<a href="https://colab.research.google.com/github/jamesyzhang10/Pneumonia/blob/main/Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from google.colab import drive
drive.mount('/content/drive')
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found GPU at: /device:GPU:0


In [72]:
train = ImageDataGenerator(rescale = 1/255)
train_set = train.flow_from_directory('/content/drive/MyDrive/Pneumonia/train', target_size = (150, 150), batch_size = 3, class_mode = 'binary')
train_set.class_indices
train_set.classes

Found 4192 images belonging to 2 classes.


array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [73]:
#Create the sequential model
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape = (150, 150, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))

model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))

#Hidden layer
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation = 'relu'))
model.add(keras.layers.Dropout(0.5))

#Output layer
model.add(keras.layers.Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 36992)           

In [74]:
#Compile
model.compile(
    optimizer = "rmsprop",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

#Train the neural network
model.fit(train_set, epochs = 3)

# Save model to file
model.save("pneumonia_classify")
reconstructed_model = tf.keras.models.load_model("pneumonia_classify")

Epoch 1/3
1398/1398 [==============================] - 49s 35ms/step - loss: 0.4180 - accuracy: 0.8411
Epoch 2/3
1398/1398 [==============================] - 49s 35ms/step - loss: 0.2053 - accuracy: 0.9407
Epoch 3/3
1398/1398 [==============================] - 49s 35ms/step - loss: 0.1736 - accuracy: 0.9528
INFO:tensorflow:Assets written to: pneumonia_classify/assets
